Downloading the brain tumor MRI dataset from kaggle

In [ ]:
import kagglehub
path = kagglehub.dataset_download("masoudnickparvar/brain-tumor-mri-dataset")
print("path to dataset ",path)


100%|██████████| 149M/149M [00:01<00:00, 89.1MB/s]

Extracting files...


path to dataset  /root/.cache/kagglehub/datasets/masoudnickparvar/brain-tumor-mri-dataset/versions/1


Install codecarbon and efficientnet

In [ ]:
!pip install torch torchvision codecarbon efficientnet_pytorch


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 57.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 71.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 516.7/516.7 kB 42.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 7.3 MB/s eta 0:00:00
   

Prepare the data for training

In [ ]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

train_path = path+"/Training"
test_path = path+"/Testing"

img_size = (256, 256)
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
])

train_dataset = datasets.ImageFolder(root=train_path, transform=transform)
test_dataset = datasets.ImageFolder(root=test_path, transform=transform)

batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

print("Classes: ", train_dataset.classes)

Classes:  ['glioma', 'meningioma', 'notumor', 'pituitary']


Loading efficientnet and setting the loss function, optimizer and learning rate

In [ ]:
import torch.nn as nn
from efficientnet_pytorch import EfficientNet

model = EfficientNet.from_pretrained('efficientnet-b0')
model._fc = nn.Linear(model._fc.in_features, 4)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b0-355c32eb.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b0-355c32eb.pth
100%|██████████| 20.4M/20.4M [00:00<00:00, 70.8MB/s]


Loaded pretrained weights for efficientnet-b0


Training, along with emissions tracking

In [ ]:
from codecarbon import EmissionsTracker
tracker = EmissionsTracker()
tracker.start()

def train_model(model, train_loader, test_loader, criterion, optimizer, epochs=5):
    for epoch in range(epochs):
        model.train()
        running_loss, correct = 0.0, 0
        total = 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * images.size(0)
            _, predicted = outputs.max(1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

        train_loss = running_loss / len(train_loader.dataset)
        train_acc = 100 * correct / total

        print(f"Epoch {epoch+1}/{epochs} - Loss: {train_loss:.4f} - Accuracy: {train_acc:.2f}%")

    evaluate_model(model, test_loader)
    tracker.stop()

def evaluate_model(model, test_loader):
    model.eval()
    correct, total = 0, 0

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = outputs.max(1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

    print(f"Test Accuracy: {100 * correct / total:.2f}%")

train_model(model, train_loader, test_loader, criterion, optimizer, epochs=10)

[codecarbon INFO @ 17:59:06] [setup] RAM Tracking...
[codecarbon INFO @ 17:59:06] [setup] CPU Tracking...
[codecarbon WARNING @ 17:59:06] No CPU tracking mode found. Falling back on CPU constant mode. 
 Linux OS detected: Please ensure RAPL files exist at \sys\class\powercap\intel-rapl to measure CPU

[codecarbon WARNING @ 17:59:07] We saw that you have a Intel(R) Xeon(R) CPU @ 2.00GHz but we don't know it. Please contact us.
[codecarbon INFO @ 17:59:07] CPU Model on constant consumption mode: Intel(R) Xeon(R) CPU @ 2.00GHz
[codecarbon INFO @ 17:59:07] [setup] GPU Tracking...
[codecarbon INFO @ 17:59:07] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 17:59:07] >>> Tracker's metadata:
[codecarbon INFO @ 17:59:07]   Platform system: Linux-6.1.85+-x86_64-with-glibc2.35
[codecarbon INFO @ 17:59:07]   Python version: 3.11.11
[codecarbon INFO @ 17:59:07]   CodeCarbon version: 2.8.3
[codecarbon INFO @ 17:59:07]   Available RAM : 12.675 GB
[codecarbon INFO @ 17:59:07]   CPU count: 2
[codeca

Epoch 1/10 - Loss: 0.2454 - Accuracy: 91.53%


[codecarbon INFO @ 18:00:22] Energy consumed for RAM : 0.000099 kWh. RAM Power : 4.7530388832092285 W
[codecarbon INFO @ 18:00:22] Energy consumed for all CPUs : 0.000885 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 18:00:22] Energy consumed for all GPUs : 0.001192 kWh. Total GPU Power : 59.572873983485806 W
[codecarbon INFO @ 18:00:22] 0.002176 kWh of electricity used since the beginning.
[codecarbon INFO @ 18:00:37] Energy consumed for RAM : 0.000119 kWh. RAM Power : 4.7530388832092285 W
[codecarbon INFO @ 18:00:37] Energy consumed for all CPUs : 0.001062 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 18:00:37] Energy consumed for all GPUs : 0.001444 kWh. Total GPU Power : 60.31340920278389 W
[codecarbon INFO @ 18:00:37] 0.002624 kWh of electricity used since the beginning.
[codecarbon INFO @ 18:00:52] Energy consumed for RAM : 0.000139 kWh. RAM Power : 4.7530388832092285 W
[codecarbon INFO @ 18:00:52] Energy consumed for all CPUs : 0.001239 kWh. Total CPU Power : 42.5 W
[codec

Epoch 2/10 - Loss: 0.0930 - Accuracy: 96.85%


[codecarbon INFO @ 18:01:37] Energy consumed for RAM : 0.000198 kWh. RAM Power : 4.7530388832092285 W
[codecarbon INFO @ 18:01:37] Energy consumed for all CPUs : 0.001770 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 18:01:37] Energy consumed for all GPUs : 0.002465 kWh. Total GPU Power : 61.37002001474245 W
[codecarbon INFO @ 18:01:37] 0.004433 kWh of electricity used since the beginning.
[codecarbon INFO @ 18:01:52] Energy consumed for RAM : 0.000218 kWh. RAM Power : 4.7530388832092285 W
[codecarbon INFO @ 18:01:52] Energy consumed for all CPUs : 0.001947 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 18:01:52] Energy consumed for all GPUs : 0.002718 kWh. Total GPU Power : 60.854904574177894 W
[codecarbon INFO @ 18:01:52] 0.004884 kWh of electricity used since the beginning.
[codecarbon INFO @ 18:02:07] Energy consumed for RAM : 0.000238 kWh. RAM Power : 4.7530388832092285 W
[codecarbon INFO @ 18:02:07] Energy consumed for all CPUs : 0.002124 kWh. Total CPU Power : 42.5 W
[codec

...

[codecarbon INFO @ 18:09:52] Energy consumed for RAM : 0.000851 kWh. RAM Power : 4.7530388832092285 W
[codecarbon INFO @ 18:09:52] Energy consumed for all CPUs : 0.007611 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 18:09:52] Energy consumed for all GPUs : 0.010866 kWh. Total GPU Power : 60.42959025196672 W
[codecarbon INFO @ 18:09:52] 0.019329 kWh of electricity used since the beginning.
[codecarbon INFO @ 18:10:07] Energy consumed for RAM : 0.000871 kWh. RAM Power : 4.7530388832092285 W
[codecarbon INFO @ 18:10:07] Energy consumed for all CPUs : 0.007788 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 18:10:07] Energy consumed for all GPUs : 0.011120 kWh. Total GPU Power : 61.028507102508826 W
[codecarbon INFO @ 18:10:07] 0.019780 kWh of electricity used since the beginning.
[codecarbon INFO @ 18:10:22] Energy consumed for RAM : 0.000891 kWh. RAM Power : 4.7530388832092285 W
[codecarbon INFO @ 18:10:22] Energy consumed for all CPUs : 0.007965 kWh. Total CPU Power : 42.5 W
[codec

Epoch 10/10 - Loss: 0.0148 - Accuracy: 99.60%


[codecarbon INFO @ 18:11:07] Energy consumed for RAM : 0.000949 kWh. RAM Power : 4.7530388832092285 W
[codecarbon INFO @ 18:11:07] Energy consumed for all CPUs : 0.008490 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 18:11:07] Energy consumed for all GPUs : 0.012095 kWh. Total GPU Power : 53.30382088059335 W
[codecarbon INFO @ 18:11:07] 0.021534 kWh of electricity used since the beginning.
[codecarbon INFO @ 18:11:07] 0.010399 g.CO2eq/s mean an estimation of 327.93116232693893 kg.CO2eq/year


Test Accuracy: 99.31%


/usr/local/lib/python3.11/dist-packages/codecarbon/output_methods/file.py:52: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, pd.DataFrame.from_records([dict(total.values)])])
